In [7]:
import json
import keras
def extractGenres(row):
    json_dict_list = json.loads(row.genres)
    genres = [d['name'].lower() for d in json_dict_list if 'name' in d]
    return " ".join(genres)

In [8]:
import pandas as pd
df = pd.read_csv('datasets/tmdb_5000_movies.csv')
X = [str(p.overview) for p in df.itertuples()]
y = [extractGenres(p).rstrip() for p in df.itertuples()]
print(len(X),len(y))

4803 4803


In [9]:
from keras.preprocessing.text import Tokenizer
#from keras.sequences import pad_sequences
Xtokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True,)
Ytokenizer = Tokenizer(filters=' ',lower=True,)

Xtokenizer.fit_on_texts(X)
plot_matrix = Xtokenizer.texts_to_matrix(X)


Ytokenizer.fit_on_texts(y)
genre_matrix = Ytokenizer.texts_to_matrix(y)

input_size = plot_matrix.shape[1]
output_size = genre_matrix.shape[1]
print(input_size,output_size)

22811 23


In [10]:
split_ratio = 0.8
split_index = int(len(X)*split_ratio)
X_train,X_test = plot_matrix[:split_index] , plot_matrix[split_index:]
y_train,y_test = genre_matrix[:split_index] , genre_matrix[split_index:]

In [17]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adadelta
model = Sequential()
model.add(Dense(512, activation='relu', input_dim=input_size))
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(output_size, activation='sigmoid'))
model.summary()

adadelta = Adadelta(lr=0.1, rho=0.95, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer=adadelta,metrics=['sparse_top_k_categorical_accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 512)               11679744  
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 23)                2967      
Total params: 11,748,375
Trainable params: 11,748,375
Non-trainable params: 0
_________________________________________________________________


In [18]:
from keras.callbacks import Callback,ModelCheckpoint,EarlyStopping
checkpoint=ModelCheckpoint(filepath=None,monitor='val_loss',mode='min',verbose=1)

In [19]:
import numpy as np
model.fit(np.array(X_train), np.array(y_train),epochs=100, batch_size=128, verbose=1,callbacks=[])

Epoch 1/100
3842/3842 [==============================] - 8s 2ms/step - loss: 8.5466 - sparse_top_k_categorical_accuracy: 0.7926
Epoch 2/100
3842/3842 [==============================] - 7s 2ms/step - loss: 7.9920 - sparse_top_k_categorical_accuracy: 1.0000
Epoch 3/100
3842/3842 [==============================] - 7s 2ms/step - loss: 7.4731 - sparse_top_k_categorical_accuracy: 1.0000
Epoch 4/100
3842/3842 [==============================] - 7s 2ms/step - loss: 7.3635 - sparse_top_k_categorical_accuracy: 1.0000
Epoch 5/100
3842/3842 [==============================] - 7s 2ms/step - loss: 7.3194 - sparse_top_k_categorical_accuracy: 1.0000
Epoch 6/100
3842/3842 [==============================] - 7s 2ms/step - loss: 7.2845 - sparse_top_k_categorical_accuracy: 1.0000
Epoch 7/100
3842/3842 [==============================] - 8s 2ms/step - loss: 7.2548 - sparse_top_k_categorical_accuracy: 1.0000
Epoch 8/100
3842/3842 [==============================] - 7s 2ms/step - loss: 7.2113 - sparse_top_k_categ

3842/3842 [==============================] - 7s 2ms/step - loss: 4.3837 - sparse_top_k_categorical_accuracy: 0.7782
Epoch 65/100
3842/3842 [==============================] - 7s 2ms/step - loss: 4.3497 - sparse_top_k_categorical_accuracy: 0.7736
Epoch 66/100
3842/3842 [==============================] - 7s 2ms/step - loss: 4.3287 - sparse_top_k_categorical_accuracy: 0.7707
Epoch 67/100
3842/3842 [==============================] - 8s 2ms/step - loss: 4.2905 - sparse_top_k_categorical_accuracy: 0.7660
Epoch 68/100
3842/3842 [==============================] - 8s 2ms/step - loss: 4.2757 - sparse_top_k_categorical_accuracy: 0.7743
Epoch 69/100
3842/3842 [==============================] - 7s 2ms/step - loss: 4.2566 - sparse_top_k_categorical_accuracy: 0.7676
Epoch 70/100
3842/3842 [==============================] - 7s 2ms/step - loss: 4.2402 - sparse_top_k_categorical_accuracy: 0.7650
Epoch 71/100
3842/3842 [==============================] - 7s 2ms/step - loss: 4.1956 - sparse_top_k_categorica

In [20]:
score = model.evaluate(np.array(X_test), np.array(y_test), verbose=0)
print("Accuracy: ", score)

Accuracy:  [5.609634281817385, 0.8720083246618107]


In [21]:
print(model.predict(np.array(X_test)))

[[3.7392962e-05 4.5298678e-03 1.4219886e-01 ... 1.0041806e-04
  4.0718842e-05 2.9475548e-05]
 [2.0476828e-06 2.9729018e-01 1.1329710e-01 ... 8.2591541e-05
  2.3299972e-05 3.3698190e-05]
 [2.2117400e-05 9.0507883e-01 1.7676065e-02 ... 2.6224409e-03
  3.8952573e-04 5.2324601e-04]
 ...
 [2.1419135e-05 2.4373114e-01 1.0262808e-02 ... 6.8459689e-04
  1.8794466e-04 1.5066075e-04]
 [6.9308487e-07 5.7306314e-01 1.0915647e-01 ... 1.1384044e-04
  1.7983399e-05 3.5880286e-05]
 [4.1873191e-06 3.2369420e-02 6.4464766e-01 ... 3.1808534e-04
  8.2639126e-05 4.1704221e-05]]


In [22]:
np.array(X_test).shape

(961, 22811)